In [3]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
os.environ['ARROW_PRE_0_15_IPC_FORMAT'] = '1'
from pyspark.pandas import read_csv
import pyspark.pandas as ps
# use tqdm for lambdas so we can see progress

In [14]:
ps.set_option('compute.default_index_type', 'distributed')
big_data = False
path ='data/100k_a.csv' if  not big_data else 'data/full_a.csv'
df = read_csv(path, names=['userId','streamId','streamerName','timeStart','timeStop'])

In [5]:
df.head()

,userId,streamId,streamerName,timeStart,timeStop
0,1,33827518864,lirik,25,26
1,1,33870289680,giantwaffle,457,460
2,1,33876317008,blacklisted86,471,472
3,1,33882713200,cohhcarnage,544,550
4,1,33885628720,lirik,586,588


In [4]:
df['interactionTime']=(df['timeStop'] - df['timeStart'])*10 # units are 10 min

In [5]:
df.head()

,userId,streamId,streamerName,timeStart,timeStop,interactionTime
0,1,33827518864,lirik,25,26,10
1,1,33870289680,giantwaffle,457,460,30
2,1,33876317008,blacklisted86,471,472,10
3,1,33882713200,cohhcarnage,544,550,60
4,1,33885628720,lirik,586,588,20


In [6]:
df= df.drop(columns=['timeStart','timeStop'])

In [7]:
tmax=df['interactionTime'].max()

In [8]:
tmin=df['interactionTime'].min()

In [9]:
df['interactionTime']= (df['interactionTime']- tmin)/(tmax-tmin)

In [10]:
df.head()

,userId,streamId,streamerName,interactionTime
0,1,33827518864,lirik,0.000000
1,1,33870289680,giantwaffle,0.011050
2,1,33876317008,blacklisted86,0.000000
3,1,33882713200,cohhcarnage,0.027624
4,1,33885628720,lirik,0.005525


In [11]:
# goal is to scale to 1-100 for implict rating/confidence
# MAYBE INVESTIGATE DIFFERENT SCALING IN FUTURE
df['interactionTime'] = (df['interactionTime'] * 99) +1

In [12]:
df.head()

,userId,streamId,streamerName,interactionTime
0,1,33827518864,lirik,1.000000
1,1,33870289680,giantwaffle,2.093923
2,1,33876317008,blacklisted86,1.000000
3,1,33882713200,cohhcarnage,3.734807
4,1,33885628720,lirik,1.546961


In [13]:
import sys
df['streamerId']=df['streamerName'].apply(lambda x: hash(x) & sys.maxsize) # prevent negative numbers

In [14]:
df.head()

,userId,streamId,streamerName,interactionTime,streamerId
0,1,33827518864,lirik,1.000000,5572103535684908044
1,1,33870289680,giantwaffle,2.093923,5677988202902416945
2,1,33876317008,blacklisted86,1.000000,4648292131931301687
3,1,33882713200,cohhcarnage,3.734807,1581951335071538364
4,1,33885628720,lirik,1.546961,5572103535684908044


In [15]:
len(df['streamerId'].unique()) == len(df['streamerName'].unique())

True

In [16]:
df = df.drop(columns=['streamerName'])

In [17]:
df.head()

,userId,streamId,interactionTime,streamerId
0,1,33827518864,1.000000,5572103535684908044
1,1,33870289680,2.093923,5677988202902416945
2,1,33876317008,1.000000,4648292131931301687
3,1,33882713200,3.734807,1581951335071538364
4,1,33885628720,1.546961,5572103535684908044


In [9]:
# better for smaller in memory data
#df.to_pandas().to_csv('data/test.csv')

In [8]:
# used to ensure no oom error with pandas
df.to_spark().coalesce(1).write.format('csv').option('header', 'true').save('data/output')

In [11]:
df2= ps.read_csv('data/mytest/part-00000-920b9e00-7728-40f1-9733-1865cb319370-c000.csv')